# To get the Bus Information from Bus Routes and Bus Stops dataset

In [ ]:
import requests
import pandas as pd
import warnings

In [ ]:
# Use URL to retrieve the bus routes data
warnings.simplefilter(action='ignore', category=FutureWarning)
url = "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes"

payload = {}
headers = {
  'AccountKey': 'TdT4Q/DhRjC9n4ZkUHrYww=='
}
response = requests.request("GET", url, headers=headers, data=payload)
bus_routes = response.json()
data = bus_routes['value']

columns = list(data[0].keys())
bus_routes = pd.DataFrame(columns=columns)
for d in data:
    bus_routes = pd.concat([bus_routes, pd.DataFrame(d, index=[0])], ignore_index=True)

bus_routes = bus_routes[bus_routes['ServiceNo'] == '10']
# bus_routes.to_csv('bus_routes.csv', index=False)
bus_routes = bus_routes[['Direction', 'StopSequence', 'BusStopCode', 'Distance']]
# bus_routes.rename(columns={'BusStopCode': 'BusStopCode_X'}, inplace=True)
bus_routes['BusStopCode_Y'] = bus_routes['BusStopCode'].shift(-1)
bus_routes.head(148)

,Direction,StopSequence,BusStopCode,Distance,BusStopCode_Y
0,1,1,75009,0,76059
1,1,2,76059,0.6,76069
2,1,3,76069,1.1,96289
3,1,4,96289,2.3,96109
4,1,5,96109,2.7,85079
...,...,...,...,...,...
143,2,70,96101,29.1,96281
144,2,71,96281,29.5,76061
145,2,72,76061,30.8,76051
146,2,73,76051,31.3,75009


In [ ]:
bus_routes['BusStopCode'] = bus_routes['BusStopCode'].astype(str)
bus_routes.to_csv('bus_routes.csv', index=False)

In [ ]:
# Use URL to retrieve the bus stops data
url = "http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4000"
payload = {}
headers = {
  'AccountKey': 'TdT4Q/DhRjC9n4ZkUHrYww=='
}
response = requests.request("GET", url, headers=headers, data=payload)
bus_stops = response.json()
data = bus_stops['value']
columns = list(data[0].keys())
bus_stops = pd.DataFrame(columns=columns)
for d in data:
    bus_stops = pd.concat([bus_stops, pd.DataFrame(d, index=[0])], ignore_index=True)
# bus_stops.to_csv('bus_stops4.csv', index=False)

In [ ]:
bus_stops = pd.read_csv('bus_stops.csv')
bus_stops['BusStopCode'] = bus_stops['BusStopCode'].astype(str)
bus_stops

,BusStopCode,RoadName,Description,Latitude,Longitude
0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
1995,96421,Tanah Merah Coast Rd,OPP CHANGI LODGE 2,1.313621,103.996550
1996,96429,Tanah Merah Coast Rd,CHANGI LODGE 2,1.313721,103.995381
1997,96431,Tanah Merah Coast Rd,OPP CHANGI NAVAL BASE,1.318287,104.015263
1998,96439,Tanah Merah Coast Rd,CHANGI NAVAL BASE,1.317708,104.015614


In [ ]:
# Merge 2 tables into one master table
pd_merge1 = pd.merge(bus_routes, bus_stops[['BusStopCode', 'Description', 'Latitude', 'Longitude']], on='BusStopCode',  how='left')
pd_merge1.rename(columns={'Latitude': 'Latitude_X', 'Longitude': 'Longitude_X'}, inplace=True)
pd_merge1

,Direction,StopSequence,BusStopCode,Distance,BusStopCode_Y,Description,Latitude_X,Longitude_X
0,1,1,75009,0,76059,Tampines Int,1.354076,103.943391
1,1,2,76059,0.6,76069,Opp Our Tampines Hub,1.352962,103.941652
2,1,3,76069,1.1,96289,Blk 147,1.348753,103.942086
3,1,4,96289,2.3,96109,Changi General Hosp,1.340055,103.948381
4,1,5,96109,2.7,85079,Opp Blk 3012,1.337371,103.950673
...,...,...,...,...,...,...,...,...
143,2,70,96101,29.1,96281,Blk 3012,1.336385,103.951057
144,2,71,96281,29.5,76061,Opp Changi General Hosp,1.339714,103.948250
145,2,72,76061,30.8,76051,Blk 938,1.348202,103.941898
146,2,73,76051,31.3,75009,Our Tampines Hub,1.353138,103.941264


In [ ]:
pd_merge = pd.merge(pd_merge1, bus_stops[['BusStopCode', 'Description', 'Latitude', 'Longitude']], left_on='BusStopCode_Y', right_on='BusStopCode',how='left')
pd_merge.rename(columns={'Latitude': 'Latitude_Y', 'Longitude': 'Longitude_Y'}, inplace=True)
pd_merge

,Direction,StopSequence,BusStopCode_x,Distance,BusStopCode_Y,Description_x,Latitude_X,Longitude_X,BusStopCode_y,Description_y,Latitude_Y,Longitude_Y
0,1,1,75009,0,76059,Tampines Int,1.354076,103.943391,76059,Opp Our Tampines Hub,1.352962,103.941652
1,1,2,76059,0.6,76069,Opp Our Tampines Hub,1.352962,103.941652,76069,Blk 147,1.348753,103.942086
2,1,3,76069,1.1,96289,Blk 147,1.348753,103.942086,96289,Changi General Hosp,1.340055,103.948381
3,1,4,96289,2.3,96109,Changi General Hosp,1.340055,103.948381,96109,Opp Blk 3012,1.337371,103.950673
4,1,5,96109,2.7,85079,Opp Blk 3012,1.337371,103.950673,85079,Flextronic,1.333949,103.950152
...,...,...,...,...,...,...,...,...,...,...,...,...
143,2,70,96101,29.1,96281,Blk 3012,1.336385,103.951057,96281,Opp Changi General Hosp,1.339714,103.948250
144,2,71,96281,29.5,76061,Opp Changi General Hosp,1.339714,103.948250,76061,Blk 938,1.348202,103.941898
145,2,72,76061,30.8,76051,Blk 938,1.348202,103.941898,76051,Our Tampines Hub,1.353138,103.941264
146,2,73,76051,31.3,75009,Our Tampines Hub,1.353138,103.941264,75009,Tampines Int,1.354076,103.943391


In [ ]:
bus_stop=pd_merge[['Direction','StopSequence','BusStopCode_x','Distance','BusStopCode_Y','Description_x','Latitude_X','Longitude_X','Latitude_Y','Longitude_Y']]
bus_stop.rename(columns={'BusStopCode_x': 'BusStopCode_X', 'Description_x': 'Description'}, inplace=True)
bus_stop

<ipython-input-52-be6542f5eb74>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_stop.rename(columns={'BusStopCode_x': 'BusStopCode_X', 'Description_x': 'Description'}, inplace=True)


,Direction,StopSequence,BusStopCode_X,Distance,BusStopCode_Y,Description,Latitude_X,Longitude_X,Latitude_Y,Longitude_Y
0,1,1,75009,0,76059,Tampines Int,1.354076,103.943391,1.352962,103.941652
1,1,2,76059,0.6,76069,Opp Our Tampines Hub,1.352962,103.941652,1.348753,103.942086
2,1,3,76069,1.1,96289,Blk 147,1.348753,103.942086,1.340055,103.948381
3,1,4,96289,2.3,96109,Changi General Hosp,1.340055,103.948381,1.337371,103.950673
4,1,5,96109,2.7,85079,Opp Blk 3012,1.337371,103.950673,1.333949,103.950152
...,...,...,...,...,...,...,...,...,...,...
143,2,70,96101,29.1,96281,Blk 3012,1.336385,103.951057,1.339714,103.948250
144,2,71,96281,29.5,76061,Opp Changi General Hosp,1.339714,103.948250,1.348202,103.941898
145,2,72,76061,30.8,76051,Blk 938,1.348202,103.941898,1.353138,103.941264
146,2,73,76051,31.3,75009,Our Tampines Hub,1.353138,103.941264,1.354076,103.943391


In [ ]:
bus_stop.to_csv('bus_stop.csv', index=False)